# Example 5: Multi-Series Forecasting at Scale

**Phase 4 Features Showcased:**
- ✅ Performance at Scale (120+ panels)
- ✅ Global Search (across product names and regions)
- ✅ Complex Filtering (category AND region AND performance)
- ✅ Multiple Views (save analysis states)
- ✅ Keyboard Navigation (rapid page browsing)
- ✅ Export (filtered subsets for reporting)
- ✅ Empty States (no matches handling)

## Use Case

Forecast 120 different time series representing sales across:
- 20 products
- 6 regions
- 3 categories

Demonstrate efficient handling of 100+ panels with rich filtering and search capabilities.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from trelliscope import Display
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

## 1. Generate 120 Time Series Forecasts

In [ ]:
# Define product × region combinations
products = [f'Product_{chr(65+i)}' for i in range(20)]  # A-T
regions = ['North', 'South', 'East', 'West', 'Central', 'Coastal']
categories = {'Electronics': products[:7], 'Apparel': products[7:14], 'Food': products[14:]}

# Map products to categories
product_category = {}
for cat, prods in categories.items():
    for prod in prods:
        product_category[prod] = cat

total_series = len(products) * len(regions)
print(f"Generating {total_series} time series forecasts...")
print(f"Products: {len(products)}")
print(f"Regions: {len(regions)}")
print(f"Total: {len(products)} × {len(regions)} = {total_series} panels\n")

In [ ]:
def create_forecast_panel(actual, forecast, product, region, mape):
    """Create compact forecast visualization for scale."""
    fig, ax = plt.subplots(figsize=(8, 4))
    
    t = np.arange(len(actual))
    ax.plot(t, actual, label='Actual', color='#2C3E50', linewidth=2, alpha=0.8)
    ax.plot(t, forecast, label='Forecast', color='#E74C3C', linewidth=1.5, linestyle='--', alpha=0.9)
    
    # Simplified confidence band
    std = np.std(actual - forecast)
    ax.fill_between(t, forecast - std, forecast + std, color='#E74C3C', alpha=0.15)
    
    ax.set_title(f'{product} - {region}\nMAPE: {mape:.1f}%', fontsize=11, fontweight='bold')
    ax.set_xlabel('Week', fontsize=9)
    ax.set_ylabel('Sales', fontsize=9)
    ax.legend(loc='upper left', fontsize=8)
    ax.grid(True, alpha=0.2, linestyle=':')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    return fig

# Generate all forecasts
data_rows = []
panel_count = 0

for product in products:
    category = product_category[product]
    
    for region in regions:
        # Generate time series with region and category effects
        n_weeks = 52
        base_level = 100
        
        # Region effects
        region_multipliers = {
            'North': 1.2, 'South': 0.9, 'East': 1.1, 
            'West': 1.3, 'Central': 0.8, 'Coastal': 1.0
        }
        base_level *= region_multipliers[region]
        
        # Category seasonality
        if category == 'Electronics':
            seasonal = 20 * np.sin(2 * np.pi * np.arange(n_weeks) / 52 - np.pi/2)  # Peak in Q4
        elif category == 'Apparel':
            seasonal = 15 * (np.sin(2 * np.pi * np.arange(n_weeks) / 26) + 
                            np.sin(2 * np.pi * np.arange(n_weeks) / 52))  # Spring/Fall
        else:  # Food
            seasonal = 10 * np.sin(2 * np.pi * np.arange(n_weeks) / 12)  # Monthly cycle
        
        trend = np.linspace(0, 10, n_weeks)
        noise = np.random.normal(0, 5, n_weeks)
        actual = base_level + trend + seasonal + noise
        
        # Generate forecast with error
        forecast_error = np.random.uniform(0.05, 0.15)  # 5-15% MAPE
        forecast = actual * (1 + np.random.normal(0, forecast_error, n_weeks))
        
        # Calculate metrics
        mape = np.mean(np.abs((actual - forecast) / actual)) * 100
        rmse = np.sqrt(np.mean((actual - forecast)**2))
        
        # Classify trend
        if np.mean(actual[-13:]) > np.mean(actual[:13]):
            trend_class = 'increasing'
        elif np.mean(actual[-13:]) < np.mean(actual[:13]):
            trend_class = 'decreasing'
        else:
            trend_class = 'flat'
        
        # Generate anomaly alerts (random)
        alert_count = np.random.poisson(2)
        
        # Create visualization
        fig = create_forecast_panel(actual, forecast, product, region, mape)
        
        data_rows.append({
            'panel': fig,
            'product': product,
            'region': region,
            'category': category,
            'forecast_mape': mape,
            'forecast_rmse': rmse,
            'trend': trend_class,
            'last_update': datetime.now() - timedelta(hours=np.random.randint(1, 168)),
            'alerts': alert_count,
            'series_id': f"{product}_{region}"
        })
        
        panel_count += 1
        if panel_count % 30 == 0:
            print(f"  Generated {panel_count}/{total_series} panels...")
        
        plt.close(fig)

print(f"\n✓ Generated {panel_count} forecast panels")

## 2. Create Trelliscope Display

In [ ]:
df = pd.DataFrame(data_rows)

print(f"\nDataFrame shape: {df.shape}")
print(f"\nSummary by category:")
print(df.groupby('category')['forecast_mape'].agg(['count', 'mean', 'min', 'max']))
print(f"\nSummary by region:")
print(df.groupby('region')['forecast_mape'].agg(['count', 'mean']))

In [ ]:
display = (
    Display(df, name="multi_series_forecasting_scale", description="Large-scale forecasting across 120 product-region combinations demonstrating performance at scale")
    .set_panel_column("panel")
    .infer_metas()
    .set_default_layout(ncol=4, nrow=3)  # 12 panels per page
    .set_default_labels(["product", "region", "forecast_mape"])
    # Best forecasts first
)


## 3. Launch Interactive Viewer

In [ ]:
from trelliscope.dash_viewer import create_dash_app

app = create_dash_app(display)

print("\n" + "="*70)
print("🚀 LAUNCHING LARGE-SCALE FORECASTING VIEWER")
print("="*70)
print(f"\n📊 Display: {display.name}")
print(f"📈 Total Series: {len(df)} (LARGE SCALE DEMO)")
print(f"🎯 Products: {df['product'].nunique()}")
print(f"🌍 Regions: {df['region'].nunique()}")
print(f"📁 Categories: {df['category'].nunique()}")
print(f"\n⚡ Performance: Efficient handling of 100+ panels")
print(f"🔍 Search: Fast global search across all metadata")
print(f"⌨️  Navigation: Keyboard shortcuts for rapid browsing")
print("\n🌐 Opening browser on http://localhost:8053...\n")

app.run(debug=False, host='127.0.0.1', port=8053)

## 4. Feature Testing Guide

### ✅ Performance at Scale (PRIMARY FOCUS)

**120 Panels**:
- 10 pages × 12 panels per page
- Test that viewer remains responsive

**Try This**:
1. **Initial Load**:
   - Observe loading state
   - Should complete in < 3s
   - Check browser memory (DevTools → Memory)

2. **Rapid Page Navigation**:
   - Press **→** (right arrow) 10 times rapidly
   - Should smoothly transition through all pages
   - No lag or freeze

3. **Filter Performance**:
   - Apply: category = "Electronics"
   - Should filter 40 panels in < 400ms
   - Add: region = "West"
   - Should refine to ~7 panels in < 300ms

4. **Search Performance**:
   - Search: "Product_A"
   - Should find 6 matches in < 200ms
   - Clear search
   - Should restore all 120 panels quickly

5. **Sort Performance**:
   - Sort: forecast_mape (ascending)
   - Should reorder 120 panels in < 400ms

**Expected**: All operations responsive, no browser freeze, smooth UX

---

### ✅ Global Search Across Dimensions

**Try This**:
1. **Product Search**:
   - Search: "Product_K" → find all regions for Product K
   - Should show 6 panels

2. **Region Search**:
   - Search: "Central" → find all products in Central region
   - Should show 20 panels

3. **Category Search**:
   - Search: "Electronics" → find all Electronics forecasts
   - Should show 42 panels (7 products × 6 regions)

4. **Trend Search**:
   - Search: "increasing" → find growing series
   - Results show series with upward trends

5. **Keyboard Shortcut**:
   - Press **/** → search gains focus
   - Type query
   - Press **Esc** → clear search

**Expected**: Search works across all text fields, fast results

---

### ✅ Complex Multi-Dimensional Filtering

**Try This**:
1. **West Region Electronics**:
   - Filter: category = "Electronics"
   - Filter: region = "West"
   - Result: 7 panels (Electronics products in West)

2. **High-Performance Forecasts**:
   - Filter: forecast_mape < 8.0 (high accuracy)
   - Filter: alerts = 0 (no anomalies)
   - Result: Best-performing, stable forecasts

3. **Problematic Series**:
   - Filter: forecast_mape > 12.0 (poor accuracy)
   - Filter: alerts > 2 (multiple anomalies)
   - Result: Series needing attention

4. **Category + Trend**:
   - Filter: category = "Apparel"
   - Filter: trend = "increasing"
   - Result: Growing Apparel markets

**Expected**: Filters combine logically (AND), counts update

---

### ✅ Multiple Saved Views

**Create These Views**:

1. **"West Region Electronics"**:
   - Filter: category = "Electronics", region = "West"
   - Sort: forecast_mape ascending
   - Labels: product, forecast_mape, alerts
   - Save as "West Region Electronics"

2. **"Underperforming Products"**:
   - Filter: forecast_mape > 12.0, alerts > 2
   - Sort: forecast_mape descending
   - Save as "Underperforming Products"

3. **"Top Performers"**:
   - Filter: forecast_mape < 8.0
   - Sort: forecast_mape ascending
   - Save as "Top Performers"

4. **"Growing Markets"**:
   - Filter: trend = "increasing", forecast_mape < 10.0
   - Sort: forecast_mape ascending
   - Save as "Growing Markets"

**Then Try**:
- Switch between views using dropdown
- Verify each view restores complete state
- Delete a view
- Verify it's removed from dropdown

**Expected**: Views preserve complex filter combinations

---

### ✅ Export Filtered Subsets

**Try This**:
1. **Export West Electronics**:
   - Load "West Region Electronics" view
   - Click "Export Data (CSV)"
   - Open CSV: should have 7 rows
   - Columns: product, region, category, forecast_mape, etc.

2. **Export Underperformers**:
   - Load "Underperforming Products" view
   - Export CSV
   - Use for improvement project

3. **Export View Config**:
   - With complex filters active
   - Click "Export View (JSON)"
   - Share JSON with colleague
   - They can recreate your analysis

**Expected**: CSV contains filtered data, JSON contains state

---

### ✅ Empty State Handling

**Try This**:
1. **Impossible Filter**:
   - Filter: forecast_mape < 1.0 (too strict)
   - AND: alerts > 10 (too high)
   - Observe empty state:
     - "No Results Found"
     - "No panels match current filters"
     - "Reset Filters" button

2. **Reset from Empty**:
   - Click "Reset Filters"
   - Verify all 120 panels return

3. **Empty Search**:
   - Search: "XYZ123" (doesn't exist)
   - Observe empty state
   - Clear search
   - Panels return

**Expected**: Helpful empty state, easy recovery

---

## 5. Analysis Examples

### Q1: Which region performs best for Electronics?

**Steps**:
1. Filter: category = "Electronics"
2. Sort: forecast_mape ascending
3. Look at top results
4. Note which regions appear most

### Q2: Which products need attention?

**Steps**:
1. Filter: forecast_mape > 12.0
2. Group mentally by product
3. Products appearing 4+ times (most regions) need work

### Q3: Are there regional patterns?

**Steps**:
1. Filter by each region individually
2. Check average MAPE per region
3. Some regions consistently harder to forecast?

### Q4: Category-specific insights?

**Steps**:
1. Filter: category = "Food"
2. Sort: forecast_mape ascending
3. Compare to "Electronics" and "Apparel"
4. Which category is easiest to forecast?

---

## Summary

This example demonstrates:
- ✅ Performance at scale (120 panels)
- ✅ Fast global search across dimensions
- ✅ Complex multi-filter combinations
- ✅ Multiple saved views for workflows
- ✅ Keyboard navigation for efficiency
- ✅ Export filtered subsets
- ✅ Empty state handling

**Key Insight**: Viewer handles 100+ panels efficiently while maintaining responsive UX!